## 인공신경망(Artificial Neural Networks)

<img src="https://i.imgur.com/EtpSmaG.png" width="600">

여기서는 딥러닝 프레임워크(Framework)인 **텐서플로우(Tensorflow)**와 그 상위 API인 **케라스(Keras)**를 사용할 것입니다.<br/>

코드는 여러 번 반복해보고 여러 데이터에 적용해보는 것이 중요합니다.<br/>
예제 이외에도 여러 데이터를 Tensorflow 와 Keras 로 풀어본다면 익숙해질 것입니다.

### Iris 데이터 분류하기

분류 문제에서 예시로 자주 활용되는 **붓꽃(Iris) 데이터**에 신경망을 적용하여 봅시다.

전체 특성 중 2개의 특성(**`sepal length, petal length`**)만 선택하여 사용하고,<br/>
150개의 데이터 중 Setosa(50), Versicolor(50)만 추출하여 100개의 데이터에 대해서 **이진 분류(Binary classification)**를 진행해 보겠습니다.

1. **먼저 필요한 패키지와 라이브러리를 불러옵니다.**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf

2. **시드(Seed)를 고정합니다.**

In [ ]:
np.random.seed(42) # numpy 랜덤 시드 고정
tf.random.set_seed(42) # tensorflow 랜덤 시드 고정

3. **Iris 데이터셋을 DataFrame 형태로 불러온 후 데이터 형태를 살펴봅니다.**

In [ ]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None)
df.head()

4. **Setosa, Versicolor 데이터만 추출하여 전처리해 줍니다.**

In [ ]:
label = df.iloc[0:100, 4].values # 4 번째 column의 값(=label)만 추출
label

타겟 레이블을 setosa = 0, versicolor = 1 로 변경해줍니다.

In [ ]:
label = np.where(label == 'Iris-setosa', 0, 1) # 값이 'Iris-setosa'인 경우 0으로 변경하고 아니면 1로 변경
label

5. **데이터가 어떤 분포를 가지고 있는지 시각화를 통해 알아봅시다.**

In [ ]:
features = df.iloc[0:100, [0,2]].values
plt.scatter(features[:50, 0], features[:50, 1], color='red', marker='o', label='setosa')
plt.scatter(features[50:100, 0], features[50:100, 1], color='blue', marker='x', label='versicolor')
plt.xlabel('sepal length')
plt.ylabel('petal length')
plt.legend(loc='upper left')
plt.show()

5. **학습 데이터셋(Train Dataset)과 시험 데이터셋(Test Dataset)으로 나누어(Split)줍니다.**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)

6. **신경망 모델을 구축하고 컴파일(complie)한 후 학습합니다.**

여기에서는 단층, 즉 **은닉층 없이 출력층으로만** 모델을 구성할 예정입니다.<br/>

In [ ]:
# Sequential: 레이어를 순차적으로 연결하여 신경망을 구성하는 방법
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

**`.compile`** 에서는 신경망에서 사용할 옵티마이저(**`optimizer`**), 손실 함수(**`loss`**), 지표(**`metrics`**)를 설정합니다.

In [ ]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

**`.fit`** 은 실제로 신경망 학습이 진행되는 부분입니다.<br/>
에포크(`epochs`)를 조정하면 학습 횟수를 조정할 수 있습니다.

In [ ]:
model.fit(X_train, y_train, epochs=30)

7. **학습한 신경망 모델을 사용하여 평가합니다.**

In [ ]:
model.predict(X_test)

In [ ]:
model.evaluate(X_test, y_test, verbose=2)

### 이탈 고객 분류하기

In [ ]:
churn_data = pd.read_csv("churn_data.csv")
category_variables = ["gender", "subscription_type", "promotion",
                      "feedback_response", "complaints", "payment_method"]
churn_data[category_variables] = churn_data[category_variables].astype("string")
churn_data2 = pd.get_dummies(churn_data, columns=category_variables, drop_first=True)

X = churn_data2.drop('churn', axis=1)
y = churn_data2['churn']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_train, X_scaled_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

첫번째 모델입니다. 3개 노드의 3개 은닉층(hidden layer) 그리고 1개의 output layer로 구성되어 있습니다.

In [ ]:
dl_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(3, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [ ]:
dl_model.compile(optimizer='sgd',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

In [ ]:
dl_model.fit(X_scaled_train, y_train, epochs=30)

모델의 성능을 살펴보시기 바랍니다.

In [ ]:
dl_model.evaluate(X_scaled_test, y_test, verbose=2)

모델 성능 향상을 위해 일부 초매개변수(hyperparameter)를 변경했습니다. 현재는 hidden layer가 5층으로 되어 있고 각 층은 8개의 노드를 가지고 있지요. 즉, 모델의 유연성이 조금 더 높아진 것입니다.

In [ ]:
dl_model2 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
dl_model2.compile(optimizer='sgd',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
dl_model2.evaluate(X_scaled_test, y_test, verbose=2)

위에서의 딥러닝은 분류(classification) 문제에 적용되었지만 회귀, 시계열, 생존분석 등 다양한 문제 상황에 응용될 수 있습니다.

* 시계열 예제: https://www.tensorflow.org/tutorials/structured_data/time_series?hl=ko
* 생존분석 예제: https://k-d-w.org/blog/2019/07/survival-analysis-for-deep-learning/

---

## 신경망 작동법 상세 설명

아래 이미지를 보면서 신경망이 어떻게 학습을 진행하는지 상상해봅시다.

<img src="https://i.imgur.com/dlGareT.gif" alt="backpropagation" width=600>

---

- **위 그림에서 설명하고 있는 과정은 다음과 같습니다.**

1. 데이터가 입력되면 신경망 각 층에서 **가중치 및 활성화 함수 연산**을 반복적으로 수행합니다.
2. 1의 과정을 모든 층에서 반복한 후에 **출력층에서 계산된 값을 출력**합니다.
3. **손실 함수**를 사용하여 **예측값(Prediction)과 실제값(Target)의 차이**를 계산합니다.
4. **경사하강법과 역전파**를 통해서 **각 가중치를 갱신**합니다.
5. 학습 중지 기준을 만족할 때까지 **1-4의 과정을 반복**합니다.

1-4의 과정을 **Iteration(이터레이션)**이라고 하며 매 Iteration 마다 가중치가 갱신됩니다.<br/>
Iteration 은 **<font color="ff6f61">순전파(1&2), 손실 계산(3), 역전파(4)</font>**로 나눠볼 수 있는데요.<br/>
먼저 비유를 통해서 신경망 학습에 대해 알아보고 각 과정에 대해서 하나씩 알아보도록 하겠습니다.

### 순전파(Forward Propagation)

위에서 알아본 것처럼 신경망은 학습 과정에서 첫 번째로 **<font color="ff6f61">순전파(Forward propagation)</font>**를 수행합니다.<br/>
순전파는 **입력층에서 입력된 신호가 은닉층의 연산을 거쳐 출력층에서 값을 내보내는 과정**인데요.

위 이미지에서 왼쪽에서 오른쪽으로 신호가 전달되는 과정을 순전파라고 하며<br/>
각 층에서의 연산 과정은 다음과 같습니다.

1. 입력층(혹은 이전 은닉층)으로부터 신호를 전달받습니다.
2. 입력된 데이터에 **가중치-편향 연산**을 수행합니다.
3. 가중합을 통해 구해진 값은 **활성화 함수**를 통해 다음 층으로 전달됩니다.

### 손실 함수(Loss function)

신경망은 손실 함수를 최소화 하는 방향으로 가중치를 갱신합니다.<br/>
그렇기 때문에 손실 함수를 잘 정의해주어야 가중치가 제대로 갱신될 수 있겠죠?

입력 데이터를 신경망에 넣어 순전파를 거치면 마지막에는 출력층을 통과한 값이 도출됩니다.<br/>
이 때 출력된 값과 그 데이터의 타겟값을 **손실 함수에 넣어 손실(Loss or Error)를 계산**하게 됩니다.

대표적인 손실 함수로는 여러분이 머신러닝 Section 에서 배웠던<br/>
**MSE(Mean-Squared Error), CEE(Cross-Entropy Error)** 등이 있습니다.

일반적으로 회귀의 손실 함수로는 **MSE** 혹은 **MAE**를,<br/>
이진 분류의 손실 함수로는 **binary_crossentropy**를,<br/>
다중 분류의 손실 함수로는 **categorical_crossentropy**와 **sparse_categorical_crossentropy**를 사용합니다.


### 역전파(Backward Propagation)

**<font color="ff6f61">역전파(Backpropagation)</font>**는 말 그대로 순전파와는 **반대 방향으로 손실(Loss or Error) 정보를 전달**해주는 과정입니다.

순전파가 **입력 신호 정보를 입력층부터 출력층까지 전달하여 값을 출력**하는 알고리즘이었다면,<br/>
역전파는 구해진 **손실 정보를 출력층부터 입력층까지 전달하여 각 가중치를 얼마나 업데이트 해야할 지를 구하는** 알고리즘입니다.

신경망은 매 반복마다 **손실(Loss)을 줄이는 방향**으로 가중치를 업데이트합니다.<br/>
그렇다면 **손실을 줄이기 위해서** 어떻게 가중치를 수정해야 할까요?

가중치 수정 방향을 결정하는 것이 바로 **<font color="ff6f61">경사 하강법(Gradient Descent, GD)</font>**입니다.<br/>
경사 하강법에 대해 알아보겠습니다.

## 경사 하강법(Gradient Descent)

아래 그림을 보면 알 수 있듯, **손실 함수 $J$ 의 경사(Gradient)가 작아지는 방향으로 업데이트** 하면 손실 함수의 값을 줄일 수 있습니다.<br/>
매 Iteration 마다 **<font color="ff6f61">해당 가중치에서의 비용 함수의 도함수(=비용 함수를 미분한 함수)를 계산</font>하여** 경사가 작아질 수 있도록 가중치를 변경합니다.

$i$ 번째 가중치인 $\theta_i$ 가 갱신되는 모습을 수식으로는 다음과 같이 나타낼 수 있습니다.

<img src="https://i.imgur.com/ic91umJ.png" height="200"/>

그림을 보면서 기울기가 양수(+)일 때에는 왜 왼쪽(-)으로 이동하게 되는지,<br/>
기울기가 음수(-)일 때에는 왜 오른쪽(+)으로 이동하게 되는지 생각해봅시다.


<img src="https://i.imgur.com/ehYYRtw.png" height="300"/>

예를 들면, 아래와 같이 가중치가 변하게 됩니다.

<img src="https://i.imgur.com/ostAP3w.gif" height="300"/>


### 다시, 역전파

그러면 각각의 가중치는 어떻게 갱신될까요?

이 과정에서 역전파의 주요 메커니즘인 **편미분**과 **Chain rule(연쇄 법칙)**이 사용됩니다.<br/>
위 식에서 볼 수 있었던 것처럼 특정 가중치 $(\theta_i)$ 에 대한 기울기는
아래 식과 같이 손실 함수를 해당 가중치로 **편미분**하여 구할 수 있습니다.

$$
\frac{\partial}{\partial \theta_i} J(\theta)
$$

<br/>

그렇다면 모든 가중치에 대한 값은 어떻게 구할 수 있을까요?<br/>
여기서 바로 **Chain rule**이 적용됩니다.<br/>
연쇄 법칙이란 아래 식과 같이 특정 변수에 대한 (편)미분 값을 다른 변수의 미분을 사용하여 나타낼 수 있는 방식입니다.

$$
\frac{\partial J(\theta)}{\partial \theta_i} = \frac{\partial J(\theta)}{\partial \theta_x} \cdot \frac{\partial \theta_x}{\partial \theta_i} = \frac{\partial J(\theta)}{\partial \theta_x} \cdot \frac{\partial \theta_x}{\partial \theta_y} \cdot \frac{\partial \theta_y}{\partial \theta_i}
$$

<br/>

연쇄 법칙을 사용하여 각 변수가 얼마나 수정되어야 할 지에 대한 정보를 전달할 수 있게 됩니다.

> ❗️ ***역전파 메커니즘에 대한 수학적인 설명은 Reference를 참조해주세요 !***


### 옵티마이저(Optimizer)

다음은 **<font color="ff6f61">옵티마이저(Optimizer)</font>** 입니다.

옵티마이저는 쉽게 말해 **<font color="ff6f61">경사를 내려가는 방법을 결정</font>**하는데요.<br/>
대표적인 옵티마이저로는 아래와 같은 것들이 있습니다. (다 외우지 않아도 됩니다!)


<img src="https://i.imgur.com/UQfpjpP.png" height="350"/>

일반적인 경사 하강법(GD)에서는 모든 입력 데이터에 대한 손실 함수의 기울기를 계산한 후에 가중치를 업데이트 하였습니다.<br/>
즉, Iteration 마다 모든 데이터를 다 사용하게 되는 것이죠.<br/>
입력 데이터가 적다면 이 방법으로도 빠르게 가중치를 금방 갱신할 수 있습니다.

하지만 실제로는 이보다 훨씬 더 큰 데이터를 다루게 되는데요.<br/>
만약 입력 데이터가 수천만개라면 모든 데이터에 대해 손실을 계산하는 과정이 굉장히 오래 걸리게 됩니다.<br/>
그러면 가중치를 수정하는데 굉장히 오랜 시간이 들어가겠죠?

#### 확률적 경사 하강법(Stochastic Gradient Descent, SGD)

그래서 등장한 것이 바로 **<font color="ff6f61">확률적 경사 하강법</font>**과 **<font color="ff6f61">미니 배치(Mini-batch) 경사 하강법</font>**입니다.

확률적 경사 하강법(SGD)은 전체 데이터에서 **하나의 데이터**를 뽑아서 신경망에 입력한 후 손실을 계산합니다.<br/>
그리고 그 손실 정보를 역전파하여 신경망의 가중치를 업데이트하게 됩니다.<br/>
다시 말하면, Iteration 마다 1개의 데이터만을 사용하는데요.

그렇기 때문에 **가중치를 빠르게 업데이트** 할 수 있다는 장점이 있습니다.<br/>
물론 확률적 경사 하강법에도 단점이 있습니다. 1개의 데이터만 보기 때문에 학습 과정에서 불안정한 경사 하강을 보인다는 점인데요.

아래 그림에서 확률적 경사 하강법(왼쪽)과 일반적인 경사 하강법(오른쪽)에서 경사 하강이 어떻게 일어나는 지의 차이를 볼 수 있습니다.

그래서 두 방법을 적절히 융화한 **미니 배치(Mini-batch) 경사 하강법**이 등장하게 되었습니다.<br/>
N개의 데이터로 미니 배치를 구성하여 해당 미니 배치를 신경망에 입력한 후 이 결과를 바탕으로 가중치를 업데이트합니다.<br/>
즉, Iteration 마다 N개(=배치 사이즈)의 데이터를 사용하게 됩니다.

일반적으로는 두 방법의 장점을 적절히 융화한 미니 배치 경사 하강법을 많이 사용합니다.

- **배치 사이즈(Batch Size)**

미니 배치 경사 하강법에서 사용하는 미니 배치의 크기를 **배치 사이즈(Batch size)** 라고 합니다.<br/>
일반적으로 배치 사이즈는 2의 배수로 설정하며, 메모리 크기가 허락한다면 큰 배치 사이즈를 쓰는 것이 학습을 안정적으로 진행할 수 있습니다.

- **여러 가지 옵티마이저(Optimizer)**

<img src="https://i.imgur.com/DYoGuTT.gif" height="400" />

여러 가지 옵티마이저 중에서 어떤 것이 가장 좋다고 말하기는 어렵습니다.<br/>
문제마다, 데이터마다 달라지기 때문에 여러 옵티마이저를 적용하면서 서로 비교해보아야 하죠.